# Basic ML: Phonocardiograms

Author: Jake Dumbauld <br>
Contact: jacobmilodumbauld@gmail.com<br>
Date: 3.15.22

## Intro

Initial modelling here - copied same model from above </br>
Found that it was not overfitting at all, began to trim some regularization and removed the callbacks. </br>
Also removed the topmost input layer of 1024 </br>
Quickly fell into overfitting, with val loss increasing after only a 2 epochs. Increased regularization adding dropout of 0.4 back to each layer </br>
Began underfitting here, so I trimmed dropout to 0.2 and removed the learning rate schedule </br> 
Quickly fell into an overfitting regime again here. </br>


Out of curiousity, I wanted to try SGD to see if I could get more robust performance on my validation & test sets.


It was around this point that I felt that I needed a more scientific approach to model evaluation and tuning. Starting from a fresh notebook.

## Imports

In [1]:
#imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
import re

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
import keras_tuner as kt

From the Keras FAQ: https://keras.io/getting_started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
- Trying to obtain reproducible results. Best I can tell scikitlearn also uses np.random() seed

In [2]:
import random as python_random

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers in a well-defined state.
python_random.seed(42)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(42)

#not sure if the below are necessary - leaving in to perhaps un-comment later.
%env PYTHONHASHSEED=0
%env CUDA_VISIBLE_DEVICES=""

env: PYTHONHASHSEED=0
env: CUDA_VISIBLE_DEVICES=""


## Helper Functions

Defining a Helper Function for Plotting Model Loss

In [3]:
def graph_model_loss(title, history):
    """
    Description:
    Graphs training vs validation loss over epochs for a given model. 
    
    History: tensorflow.python.keras.callbacks.History object
    Title: str
    """ 
    plt.figure(figsize=(12,8))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(title,size=24)
    plt.ylabel('Loss',size=16)
    plt.xlabel('Epoch',size=16)
    plt.legend(['Train', 'Validation'])
    plt.show()

Defining a helper function to evaluate train & test accuracies

In [4]:
def evaluate_model(model, history):
    """
    Description:
    Outputs model train & test accuracies for currently defined train and test set variables.
    
    model: tensorflow model,
    history: tensorflow.python.keras.callbacks.History object
    """
    # Evaluate the network
    train_accuracy = history.history["binary_accuracy"][-1]
    result = model.evaluate(X_test,y_test, verbose=1)

    print(f"Train Accuracy: {np.round(train_accuracy, 6)*100}%")
    print(f"Test Accuracy: {np.round(result[1], 6)*100}%")

Model structure to be used throughout the notebook - FULL WRITE-UP TO COME

In [5]:
def build_model(hp):
    model = keras.Sequential()
    #flattening input
    model.add(Flatten())
    
    for i in range(hp.Int('layers', 2, 4)):
        model.add(
            Dense(
            #Tuning the number of units in my input layer.
            units=hp.Int("units" + str(i), min_value=32, max_value=1024, step=64),
            kernel_regularizer=regularizers.l2(0.001),
            activation="relu"
            )
        )
        #Tuning whether or not to use dropout.
        if hp.Boolean("dropout" + str(i)):
            model.add(Dropout(rate=0.25))

        #Adding batch normalization
        if hp.Boolean("normalization" + str(i)):
            model.add(BatchNormalization())

    #output layer
    model.add(Dense(1, activation="sigmoid"))
    
    #defining learning rate
    lr_schedule = keras.optimizers.schedules.InverseTimeDecay(
                      #tuning initial learning rate
                      initial_learning_rate=hp.Float("starting_learning_rate", min_value=1e-4, max_value=1e-2, sampling="log"),
                      decay_steps=1.0,
                      decay_rate=0.1
                  )
    model.compile(
        #Optimizer
        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule),
        #Loss
        loss=keras.losses.BinaryCrossentropy(),
        #Metrics
        metrics=[keras.metrics.BinaryAccuracy()]
    )
    return model

build_model(kt.HyperParameters())

2022-03-30 21:50:43.310818: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Building a Simple Feed Forward Network

### Raw Signal Data

In [ ]:
raw = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/signal_murmur_presimple_4k.npy', allow_pickle=True)

In [ ]:
y = raw[:,0] #murmurs are just the first column
X = raw[:,1:]

In [ ]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.3)

In [ ]:
sequential_raw_signal_data_tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=10,
    seed=42,
    overwrite=True,
    directory='/Users/jmd/Documents/BOOTCAMP/Capstone/kerastune_searches',
    project_name='sequential_raw_signal_data'
)

sequential_raw_signal_data_tuner_tuner.search(X_train, y_train, epochs=100, validation_data=(X_val,y_val), callbacks=[es_callback])

In [ ]:
sequential_raw_signal_data_tuner.results_summary(num_trials=1)

In [ ]:
# Get the best hyperparameters.
best_hps = sequential_raw_signal_data_tuner.get_best_hyperparameters()
# Build the model with the best hp.
sequential_raw_signal_data = build_model(best_hps[0])

sequential_raw_signal_data_history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_val,y_val), callbacks=[es_callback])

In [ ]:
sequential_raw_signal_data.summary()

In [ ]:
evaluate_model(sequential_raw_signal_data, sequential_raw_signal_data_history)

In [ ]:
graph_model_loss('Raw Signal Data, No Patient Information', sequential_raw_signal_data_history)

In [ ]:
#saving model
sequential_raw_signal_data.save('/Users/jmd/Documents/BOOTCAMP/Capstone/neural_nets/sequential_raw_signal_no_patient', overwrite=False)

### MFCC w/o Patient Info: FFN

In [ ]:
X = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/MFCCs_noPatient.npy', allow_pickle=True)
y = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/target_array.npy', allow_pickle=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.3)

In [ ]:
sequential_MFCC_no_patient_tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=50,
    seed=42,
    overwrite=True,
    directory='/Users/jmd/Documents/BOOTCAMP/Capstone/kerastune_searches',
    project_name='sequential_MFCC_no_patient'
)

sequential_MFCC_no_patient_tuner.search(X_train, y_train, epochs=100, validation_data=(X_val,y_val), callbacks=[es_callback])

In [ ]:
sequential_MFCC_no_patient_tuner.results_summary(num_trials=1)

In [ ]:
# Get the best hyperparameters.
best_hps = sequential_MFCC_no_patient_tuner.get_best_hyperparameters()
# Build the model with the best hp.
sequential_MFCC_no_patient = build_model(best_hps[0])

sequential_MFCC_no_patient_history = sequential_MFCC_no_patient.fit(X_train, y_train, epochs=1000, validation_data=(X_val,y_val), callbacks=[es_callback])

In [ ]:
sequential_MFCC_no_patient.summary()

In [ ]:
evaluate_model(sequential_MFCC_no_patient, sequential_MFCC_no_patient_history)

In [ ]:
graph_model_loss('MFCC w/o Patient Information', sequential_MFCC_no_patient_history)

In [ ]:
#saving
sequential_MFCC_no_patient.save('/Users/jmd/Documents/BOOTCAMP/Capstone/neural_nets/sequential_MFCC_no_patient', overwrite=True)

below approach was BS and didn't work. commenting out for later.

In [ ]:
# # Create a new sequential model
# FNN_MFCC_n_pt = keras.Sequential()

# # Declare the hidden layers
# FNN_MFCC_n_pt.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.001), activation="relu"))
# #model.add(layers.Dropout(0.4))
# FNN_MFCC_n_pt.add(layers.BatchNormalization())  

# FNN_MFCC_n_pt.add(layers.Dense(128, kernel_regularizer=regularizers.l2(0.001), activation="relu"))
# #model.add(layers.Dropout(0.4))
# FNN_MFCC_n_pt.add(layers.BatchNormalization())  

# FNN_MFCC_n_pt.add(layers.Dense(32, kernel_regularizer=regularizers.l2(0.001), activation="relu"))
# #model.add(layers.Dropout(0.4))

# # Declare the output layer
# FNN_MFCC_n_pt.add(layers.Dense(1, kernel_regularizer=regularizers.l2(0.001), activation="sigmoid"))

# #declaring learning rate schedule
# lr_schedule = keras.optimizers.schedules.InverseTimeDecay(0.01, decay_steps=1.0, decay_rate=0.1)


# FNN_MFCC_n_pt.compile(
#     # Optimizer
#     optimizer=keras.optimizers.Adam(lr_schedule),  
#     # Loss function to minimize
#     loss=keras.losses.BinaryCrossentropy(),
#     # Metric used to evaluate model
#     metrics=[keras.metrics.BinaryAccuracy()]
# )

# es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# history2 = FNN_MFCC_n_pt.fit(X_train, y_train, epochs=1000, verbose=1, validation_split=0.3, callbacks=[es_callback])

# plt.plot(history2.history['loss'])
# plt.plot(history2.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'])
# plt.show()

# # Evaluate the network
# train_accuracy = history2.history["binary_accuracy"][-1]
# result = FNN_MFCC_n_pt.evaluate(X_test,y_test, verbose=1)

# print(f"Train Accuracy: {train_accuracy:.4f}")
# print(f"Test Accuracy: {result[1]:.4f}")

# FNN_MFCC_n_pt.summary()

# #saving model
# FNN_MFCC_n_pt.save('/Users/jmd/Documents/BOOTCAMP/Capstone/neural_nets/FFN_MFCC_no_patient', overwrite=False)

### MFCC w/ Patient Info: FFN

In [ ]:
X = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/MFCCs_withPatient.npy', allow_pickle=True)
y = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/target_array.npy', allow_pickle=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.3)

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
sequential_MFCC_with_patient_tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=50,
    seed=42,
    overwrite=True,
    directory='/Users/jmd/Documents/BOOTCAMP/Capstone/kerastune_searches',
    project_name='sequential_MFCC_with_patient'
)

sequential_MFCC_with_patient_tuner.search(X_train, y_train, epochs=100, validation_data=(X_val,y_val), callbacks=[es_callback])

In [ ]:
sequential_MFCC_with_patient_tuner.results_summary(num_trials=1)

In [ ]:
# Get the best hyperparameters.
best_hps = sequential_MFCC_with_patient_tuner.get_best_hyperparameters()
# Build the model with the best hp.
sequential_MFCC_with_patient = build_model(best_hps[0])

sequential_MFCC_with_patient_history = sequential_MFCC_with_patient.fit(X_train, y_train, epochs=1000, validation_data=(X_val,y_val), callbacks=[es_callback])

In [ ]:
sequential_MFCC_with_patient.summary()

In [ ]:
evaluate_model(sequential_MFCC_with_patient, sequential_MFCC_with_patient_history)

In [ ]:
graph_model_loss('MFCC w/o Patient Information', sequential_MFCC_with_patient_history)

In [ ]:
#saving model
sequential_MFCC_with_patient.save('/Users/jmd/Documents/BOOTCAMP/Capstone/neural_nets/sequential_MFCC_with_patient', overwrite=False)

---

## Setting up for CNN w/ and w/o MFCC

In [6]:
def build_CNN_model(hp):
    model = keras.Sequential()
    
    for i in range(hp.Int('conv_layers', 1, 3)):
        model.add(
            Conv2D(
            #Tuning the number of units in my input layer.
            filters=hp.Int("filters" + str(i), min_value=32, max_value=128, step=16),
            kernel_size=(3,3),
            activation="relu"
            )
        )
        #Tuning whether or not to use dropout.
        if hp.Boolean("conv_dropout" + str(i)):
            model.add(layers.Dropout(rate=0.25))

    model.add(Flatten())
    
    for i in range(hp.Int('dense_layers', 1, 3)):
        model.add(
            Dense(
            #Tuning the number of units in my input layer.
            units=hp.Int("units" + str(i), min_value=32, max_value=256, step=32),
            activation="relu"
            )
        )
        #Tuning whether or not to use dropout.
        if hp.Boolean("dropout" + str(i)):
            model.add(layers.Dropout(rate=0.25))

        #Adding batch normalization
        if hp.Boolean("normalization" + str(i)):
            model.add(layers.BatchNormalization())

    #output layer
    model.add(Dense(1, activation="sigmoid"))
    
    #defining learning rate
    lr_schedule = keras.optimizers.schedules.InverseTimeDecay(
                      #tuning initial learning rate
                      initial_learning_rate=hp.Float("starting_learning_rate", min_value=1e-4, max_value=1e-2, sampling="log"),
                      decay_steps=1.0,
                      decay_rate=0.1
                  )
    model.compile(
        #Optimizer
        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule),
        #Loss
        loss=keras.losses.BinaryCrossentropy(),
        #Metrics
        metrics=[keras.metrics.BinaryAccuracy()]
    )
    return model

build_CNN_model(kt.HyperParameters())

### CNN w/o Patient Info

In [7]:
X = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/MFCCs_noPatient.npy', allow_pickle=True)
y = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/target_array.npy', allow_pickle=True)

In [8]:
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.3)

In [10]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [11]:
CNN_MFCC_no_patient_tuner = kt.BayesianOptimization(
    hypermodel=build_CNN_model,
    objective="val_loss",
    max_trials=250,
    seed=42,
    overwrite=True,
    directory='/Users/jmd/Documents/BOOTCAMP/Capstone/kerastune_searches',
    project_name='CNN_MFCC_no_patient'
)

In [ ]:
CNN_MFCC_no_patient_tuner.search(X_train, y_train, epochs=100, validation_data=(X_val,y_val), callbacks=[es_callback])

In [ ]:
CNN_MFCC_no_patient_tuner.results_summary(num_trials=1)

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# Get the best hyperparameters.
best_hps = CNN_MFCC_no_patient_tuner.get_best_hyperparameters()
# Build the model with the best hp.
CNN_MFCC_no_patient = build_CNN_model(best_hps[0])

CNN_MFCC_no_patient_history = CNN_MFCC_no_patient.fit(X_train, y_train, epochs=5, validation_data=(X_val,y_val), callbacks=[es_callback])

In [ ]:
CNN_MFCC_no_patient.summary()

In [ ]:
evaluate_model(CNN_MFCC_no_patient, CNN_MFCC_no_patient_history)

In [ ]:
graph_model_loss('CNN MFCC w/o Patient Information', CNN_MFCC_no_patient_history)

In [ ]:
#saving model
CNN_MFCC_no_patient_tuner.save('/Users/jmd/Documents/BOOTCAMP/Capstone/neural_nets/CNN_MFCC_no_patient_tuner', overwrite=False)

## Framework

How to Approach the Problem:
- Current plan:
    - Intend to initially fit an RNN to the raw, unpadded signal data and see how it fares. 
    - Also fit a CNN to the MFCC data and see how that fares
    - Stack the outputs of those two with a third CNN that incorporates the patient information provided in the annotations.
    
- Questions:
    - Does this plan make sense?
    - How should I go about optimizing my parameters & deciding on network architecture?
    - 

In [ ]:
# use glob to extract files. 

- do LTSM
- 1D CNN 


- Trim sampling rate to 25% of what it is now.

RNN approach, concatenate patient information into a 1D vector to feed into the model

Look into an ensemble/aggregation model. 

Can incorporate patient info into CNN as well, just need to pad the vector

Look into training accelerometer data on LTSMs. 

Look into varying model hyperparameters based on frequency of the data.

Look into example architectures for CNN that are used on MFCCs

make sure to train a dumb feed forward model

Try 1024 for an input layer. 

1. Train a simple feed forward on the padded and transformed data. 
- MFCC data with patient demo info into feed forward. 
2. RNN, with patient info
- First train without, then train with. 
3. CNN on MFCCs with patient info. 
- First train out, then train with. 